In [6]:
import os
import yaml

dir_path = 'D:/data/chatbot'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

In [10]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

print( tf.VERSION )


1.13.1


In [11]:

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions ) 
tokenized_eng_lines = tokenizer.texts_to_sequences(questions) 

length_list = list()
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'English max length is {}'.format( max_input_length ))

padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_eng_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1
print( 'Number of English tokens = {}'.format( num_eng_tokens))

English max length is 22
Encoder input data shape -> (566, 22)
Number of English tokens = 518


In [29]:

french_lines = list()
for line in answers:
    french_lines.append( '<START> ' + str(line) + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( french_lines ) 
tokenized_french_lines = tokenizer.texts_to_sequences( french_lines ) 

length_list = list()
for token_seq in tokenized_french_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'French max length is {}'.format( max_output_length ))

padded_french_lines = preprocessing.sequence.pad_sequences( tokenized_french_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_french_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

french_word_dict = tokenizer.word_index
num_french_tokens = len( french_word_dict )+1
print( 'Number of French tokens = {}'.format( num_french_tokens))


French max length is 74
Decoder input data shape -> (566, 74)
Number of French tokens = 1693


In [30]:
decoder_target_data = list()
for token_seq in tokenized_french_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_french_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_french_lines = utils.to_categorical( padded_french_lines , num_french_tokens )
decoder_target_data = np.array( onehot_french_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))


Decoder target data shape -> (566, 74, 1693)


In [32]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_eng_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_french_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_french_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    132608      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embeddin

In [34]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=10 ) 
model.save( 'model.h5' ) 

Epoch 1/10
566/566 [==============================] - 125s 221ms/sample - loss: 7.4275
Epoch 2/10
566/566 [==============================] - 40s 71ms/sample - loss: 7.4092
Epoch 3/10
566/566 [==============================] - 38s 66ms/sample - loss: 7.3409
Epoch 4/10
566/566 [==============================] - 41s 73ms/sample - loss: 6.9985
Epoch 5/10
566/566 [==============================] - 38s 67ms/sample - loss: 6.4134
Epoch 6/10
566/566 [==============================] - 43s 76ms/sample - loss: 5.9768
Epoch 7/10
566/566 [==============================] - 42s 74ms/sample - loss: 5.9047
Epoch 8/10
566/566 [==============================] - 61s 108ms/sample - loss: 5.8020
Epoch 9/10
566/566 [==============================] - 38s 67ms/sample - loss: 5.7304
Epoch 10/10
566/566 [==============================] - 34s 61ms/sample - loss: 5.7161


In [38]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 256 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 256 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [39]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( eng_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')


In [1]:

enc_model , dec_model = make_inference_models()

enc_model.save( 'enc_model.h5' ) 
dec_model.save( 'dec_model.h5' ) 
model.save( 'model.h5' ) 

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter eng sentence : ' ) ) )
    #states_values = enc_model.predict( encoder_input_data[ epoch ] )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = french_word_dict['end']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in french_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )
    


NameError: name 'make_inference_models' is not defined

In [46]:
french_word_dict

{'end': 1,
 'start': 2,
 'i': 3,
 'a': 4,
 'the': 5,
 'you': 6,
 'of': 7,
 'to': 8,
 'and': 9,
 'is': 10,
 'not': 11,
 'do': 12,
 'that': 13,
 'in': 14,
 'what': 15,
 'am': 16,
 'as': 17,
 'it': 18,
 'have': 19,
 'are': 20,
 'my': 21,
 'get': 22,
 "i'm": 23,
 'when': 24,
 'be': 25,
 'me': 26,
 'can': 27,
 'an': 28,
 'feel': 29,
 'by': 30,
 'for': 31,
 'or': 32,
 'cross': 33,
 'with': 34,
 'no': 35,
 "don't": 36,
 'software': 37,
 'about': 38,
 'on': 39,
 'all': 40,
 'like': 41,
 'but': 42,
 'very': 43,
 'he': 44,
 'think': 45,
 'how': 46,
 'at': 47,
 'which': 48,
 'so': 49,
 'computer': 50,
 'one': 51,
 'your': 52,
 'was': 53,
 'much': 54,
 'any': 55,
 'could': 56,
 'from': 57,
 'human': 58,
 'say': 59,
 'really': 60,
 'we': 61,
 'been': 62,
 'emotion': 63,
 'if': 64,
 "that's": 65,
 'more': 66,
 'just': 67,
 'right': 68,
 'why': 69,
 'yet': 70,
 'feeling': 71,
 'said': 72,
 'hard': 73,
 'too': 74,
 'than': 75,
 'time': 76,
 'would': 77,
 'yes': 78,
 'should': 79,
 'only': 80,
 'know':

In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [ ]:
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")